In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, mean_squared_error, roc_auc_score
import numpy as np
import joblib

In [2]:
data = pd.read_parquet('./input/creditos_hist.parquet') # Base de la Central de Deudores histórica

In [3]:
data = data.sort_values(by=["identificacion", "entidad", "periodo"])

data["buen_historial"] = data.groupby(["identificacion", "entidad"])["situacion"].expanding().min().reset_index(level=[0,1], drop=True).eq(0).astype(int)

# La literatura indica que también importa la duración de la relación empresa-banco, por lo que contamos la cantidad 
# de periodos que aparece cada par: empresa-banco
data["n_periodos"] = data.groupby(["identificacion", "entidad"]).cumcount()

In [4]:
# Eliminamos las situaciones 0, que indican que el crédito ya fue pagado
data = data.loc[data['situacion'] != 0]
data = data.drop('denominacion', axis = 1) # Elimino la columna con las razones sociales para ahorrar RAM

In [5]:
# También puede importar la cantidad de meses en los cuales una relación entre banco y empresa se encuentra "activa"
data['periodo_int'] = data['periodo'].astype('int')

# Calcular la diferencia entre el periodo actual y el anterior dentro de cada grupo
data["diff"] = data.groupby(["identificacion", "entidad"])["periodo_int"].diff()

# Identificar nuevas secuencias (cuando hay un salto en los periodos)
data["nueva_secuencia"] = (data["diff"] != 1) & data["diff"].notna()

# Contar la cantidad de periodos consecutivos en los que la relación estuvo activa
data["n_periodos_activo"] = data.groupby(["identificacion", "entidad"])["nueva_secuencia"].cumsum()

# Eliminar la columna auxiliar
data.drop(columns=["diff", "nueva_secuencia", 'periodo_int'], inplace=True)

In [6]:
data = data.sort_values(by=['identificacion', 'periodo'], ascending= False) # Ordenamos de acuerdo a cada empresa y periodo

In [7]:
# Una variable que puede ser de interés es cuantos créditos tiene una empresa en un momento dado del tiempo
counts = data.groupby(['identificacion', 'periodo']).size().reset_index(name='n_creditos')

# También nos interesa cuanta plata debe una empresa en cada momento dado
sums = data.groupby(['identificacion', 'periodo'], as_index=True)['monto'].sum().reset_index(name='sum_montos')

# Definimos como default cuando el crédito se encuentra en situación 4 o 5, por lo que creamos la dummy de default
# Esta es nuestra variable dependiente
data['default'] = (data['situacion'] >= 4).astype(int)

In [8]:
# Queremos predecir el default el periodo siguiente
data['default_lag'] = data.groupby(['identificacion', 'entidad'])['default'].shift(1) # Lag a la variable default
data = data.dropna(subset=['default_lag']) # Eliminamos las observaciones que no tienen variable dependiente
data['default_lag'] = data['default_lag'].astype(int) # Cambio el dtype de la variable de interés

In [9]:
data["sin_historial"] = (
    data.groupby("identificacion")["periodo"]
    .transform("rank", method="first") == 1).astype(int)

# Puede influir el hecho de que la empresa no tenga historial crediticio
# Creamos una variable que indica si es la primera vez que aparece en la base

In [10]:
data["prev_default_general"] = (
    data.groupby("identificacion")["default"].expanding().max().reset_index(level=0, drop=True).astype(int)
)

data["prev_default_entidad"] = (
    data.groupby(["identificacion", 'entidad'])["default"].expanding().max().reset_index(level=[0,1], drop=True).astype(int)
)

In [11]:
default_max = (
    data.groupby(["identificacion", "entidad"])["default"]
    .expanding()
    .max()
    .reset_index(level=[0, 1], drop=True)
)

data["buen_historial"] = ((data['buen_historial'] == 1) & (default_max == 0)).astype(int)

del default_max

In [12]:
# Agregamos las nuevas variables al dataframe
data = data.merge(counts, on=['identificacion', 'periodo'], how='left')
data = data.merge(sums, on=['identificacion', 'periodo'], how='left')

del sums, counts # Para ahorrar RAM

In [13]:
# Por último, la literatura también resalta que la intensidad de la relación empresa-banco es relevante
# Usamos como proxy para la intensidad la proporción del monto adeudado con un banco sobre el total adeudado
data['monto_relativo'] = data['monto'] / data['sum_montos']

In [14]:
# Convertir "periodo" a formato de fecha
data["periodo_fecha"] = pd.to_datetime(data["periodo"].astype(str), format="%Y%m")

# Calcular la cantidad de meses transcurridos desde el primer periodo
min_periodo = data["periodo_fecha"].min()
data["desde_inicio"] = ((data["periodo_fecha"].dt.year - min_periodo.year) * 12 + 
                              (data["periodo_fecha"].dt.month - min_periodo.month))

In [15]:
data = data.loc[data['periodo'] > '202310'] # Entrenamos el modelo solo con datos a partir de octubre de 2023

In [16]:
# Elijo aleatoriamente un porcentaje de las empresas de la población
np.random.seed(42)
cuits = data['identificacion'].unique()
moneda = np.random.binomial(1, 0.2, len(cuits)) # Es como tirar una moneda sesgada para que agarre un porcentaje arbitrario de las empresas 
cuits_aleatorios = cuits[moneda == 1] # Estos son los cuits con los que me voy a quedar
data = data.loc[data['identificacion'].isin(cuits_aleatorios)] # Me quedo unicamente con las obs que tienen un cuit dentro de los seleccionados aleatoriamente

del cuits, moneda, cuits_aleatorios

In [2]:
pv = pd.read_parquet('./input/principales_variables.parquet') # Datos de principales variables monetarias provenientes de la API del BCRA
pv.reset_index(inplace= True) # el index es la fecha, así que lo paso a columna
pv['fecha'] = pd.to_datetime(pv['fecha']) # paso la nueva columna al formato correcto
pv['periodo'] = pv['fecha'].dt.strftime('%Y%m') # armo una variable llamada periodo igual a la que tengo en los datos de la Central de Deudores
pv = pv.drop('fecha', axis = 1).groupby('periodo').agg(['mean', 'std']) # elimino la de "fecha" porque no me interesan los datos diarios
# Me quedo únicamente con los promedios por mes y también calculo el desvío estándar
pv.columns = ['_'.join(col).strip() for col in pv.columns] # Renombro las columnas para que sea más prolijo
pv.reset_index(inplace= True) # Vuelvo a agregar la columna periodo
pv = pv.loc[pv['periodo'].astype(int) <= 202411] # En la Central de Deudores tenemos datos hasta 202410
pv = pv.dropna(axis = 1) # Elimino las columnas con NAs

In [3]:
pv = pv.sort_values("periodo")

pv["inflacion_acumulada"] = (1 + pv["Inflación mensual (variación en %)_mean"]/100).cumprod() # Calculo la inflación acumulada

columnas = pv.columns.tolist()
tasas = [3, 4, 5, 6, 28, 29, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53] # Estos son los índices de las columnas que tienen tasas
excluir = [0, 11, 12, 25, 26, 27, 34, 35, 58, 59, 60, 61, 62, 63, 64] # Estos son los índices de las columnas que ya están en valores reales

columnas_excluir = [columnas[i] for i in excluir]
columnas_tasas = [columnas[i] for i in tasas]

columnas_nominales = [col for col in pv.columns if col not in columnas_excluir and col not in columnas_tasas] # El resto son columnas en valores nominales

for col in columnas_nominales:
    nombre_col_real = f"{col}_real"
    pv[nombre_col_real] = pv[col] / pv['inflacion_acumulada'] # Pasamos los valores de las columnas nominales a valores reales del primer periodo

for col in columnas_tasas:
    nombre_col_real = f"{col}_real"
    pv[col] = pv[col]/100
    pv[nombre_col_real] = pv[col]/pv['inflacion_acumulada'] # A las tasas también las pasamos a valores reales, pero antes las dividimos pro 100
    
    
del col, columnas, columnas_excluir, columnas_tasas, columnas_nominales, tasas, excluir, nombre_col_real

In [18]:
data = data.merge(pv, on = 'periodo', how = 'left') # Junto las principales variables monetarias con la Central de Deudores

In [19]:
data['monto_real'] = data['monto']/data['inflacion_acumulada'] # Pasamos el monto del outstanding a valor real
data['sum_montos_real'] = data['sum_montos']/data['inflacion_acumulada'] # Lo mismo con el monto total adeudado
data.drop('inflacion_acumulada', axis = 1, inplace = True)

In [4]:
emae = pd.read_excel('./input/sh_emae_mensual_base2004.xls', index_col=[0,1]) # EMAE mensual
meses_a_numeros = {
    'Enero': '01', 'Febrero': '02', 'Marzo': '03', 'Abril': '04',
    'Mayo': '05', 'Junio': '06', 'Julio': '07', 'Agosto': '08',
    'Septiembre': '09', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'
} # Diccionario para pasar los meses a números
emae = emae.reset_index()
emae['level_1'] = emae['level_1'].map(meses_a_numeros) # Pasamos los meses a números
emae['periodo'] = emae['Período'].astype(str) + emae['level_1'] # Armamos la variable periodo
emae = emae.drop(columns=['level_1', 'Período'])

del meses_a_numeros

In [21]:
data = data.merge(emae, on = 'periodo', how= 'left') # Juntamos las bases

In [22]:
arca = pd.read_parquet('./input/constancia_inscripcion.parquet') # Cargo los datos de la constancia de inscripción de ARCA

In [23]:
cuits_arca = set(arca['identificacion']) # Los cuits que están en la base de ARCA
cuits_bcra = set(data['identificacion']) # Los cuits que están en la Central de Deudores

faltan = list(cuits_bcra - cuits_arca) # Si están en la Central de Deudores pero no en ARCA es por alguna irregularidad en ARCA

# Creemos que tener irregularidades en ARCA puede ser buen predictor de default
data['sin_arca'] = (data['identificacion'].isin(faltan)).astype(int)  # Creamos la variable

del cuits_arca, cuits_bcra, faltan

In [11]:
# Ponemos bien el tipo de dato para las columnas categóricas, así el get_dummies funciona bien
data['identificacion'] = data['identificacion'].astype('category')
data['entidad'] = data['entidad'].astype('category')
data['situacion'] = data['situacion'].astype('category')
data['default'] = data['default'].astype('category')
data['periodo'] = data['periodo'].astype('category')
data['default_lag'] = data['default_lag'].astype('category')
data['prev_default_entidad'] = data['prev_default_entidad'].astype('category')
data['prev_default_general'] = data['prev_default_general'].astype('category')
data['sin_arca'] = data['sin_arca'].astype('category')
data['buen_historial'] = data['buen_historial'].astype('category')
data['sin_historial'] = data['sin_historial'].astype('category')

In [ ]:
data.to_parquet('./input/data_limpio.parquet')

In [6]:
data = pd.read_parquet('./input/data_limpio.parquet')

Para hacer cross validation, tenemos que tener en cuenta que tenemos un panel. Por lo tanto, vamos a entrenar el modelo con datos del pasado y evaluarlo con datos del futuro

In [ ]:
# Cross Validation
data.sort_values(by=['periodo'], ascending= True, inplace = True) # Ordeno de acuerdo a la fecha
data = data.reset_index().drop(columns= 'index')
split_index = int(len(data) * 0.8) # El 80% de las observaciones más antiguas
train_indices = data.iloc[:split_index].index # Estos son los índices con los que después voy a separar en test y train
test_indices = data.iloc[split_index:].index

In [8]:
# Variable dependiente
Y = data['default_lag']

In [9]:
columnas = ['desde_inicio', 'monto', 'n_creditos', 'sum_montos', 'n_periodos', 'n_periodos_activo', 'monto_relativo', 'sin_arca', 'default', 'prev_default_entidad', 'prev_default_general', 'sin_historial', 'monto_real', 'sum_montos_real', 'buen_historial'] # Algunas de las variables independientes del modelo
# Si agrego efectos fijos por tiempo, "periodo" va después de "sin_historial"
pv.set_index('periodo', inplace= True)
pv.drop('inflacion_acumulada', axis = 1, inplace = True)
columnas.extend(pv.columns) # Todas las columnas de las principales variables monetarias
emae.set_index('periodo', inplace= True)
columnas.extend(emae.columns) # Todas las columnas de emae

In [10]:
with open('./input/columnas.txt', 'w', encoding = 'utf-8') as f:
    for col in columnas:
        f.write(f'{col}\n')

In [12]:
data.columns = [col.replace('\n', ' ') for col in data.columns]
columnas = [col.replace('\n', ' ') for col in columnas]
X = pd.get_dummies(data[columnas], drop_first=True) # Meto las columnas en get_dummies

In [14]:
X_cambio_index = X.loc[X['default_1'] != Y].index # Estas son las observaciones que cambiaron de estado entre t y t+1
X = X.drop('default_1', axis = 1) # Si no queremos que sea autorregresivo droppeamos la variable laggeada

# Separo en entrenamiento y test
X_train = X.loc[train_indices]
Y_train = Y.loc[train_indices]
X_test = X.loc[test_indices]
Y_test = Y.loc[test_indices]

# También identificamos las observaciones en el conjunto de entrenamiento que cambiaron de estado
X_test_cambio_index =list(set(X_test.index) & set(X_cambio_index))
Y_test_cambio_index = list(set(Y_test.index) & set(X_cambio_index))

X_test_cambio = X_test.loc[X_test_cambio_index]
Y_test_cambio = Y_test.loc[Y_test_cambio_index]

del train_indices, test_indices, X_test_cambio_index, Y_test_cambio_index, split_index, X_cambio_index

In [31]:
# Función para evaluar los modelos
def eval(model, X_test, Y_test, linear = None):
    y_pred = model.predict(X_test)
    
    if linear:
        y_pred = np.where(y_pred >= 0.5, 1, 0)
    
    cm = confusion_matrix(Y_test, y_pred)
    
    #precision = precision_score(Y_test, y_pred)
    #recall = recall_score(Y_test, y_pred)
    f1 = f1_score(Y_test, y_pred)
    accuracy = accuracy_score(Y_test, y_pred)
    mse = mean_squared_error(Y_test, y_pred)
    auc = roc_auc_score(Y_test, y_pred)
    
    print(cm)
    #print(f'La precisión es: {precision}')
    #print(f'El recall es: {recall}')
    print(f'El f1 es: {f1}')
    print(f'El accuracy es: {accuracy}')
    print(f'El MSE es: {mse}')
    print(f'El AUC es: {auc}')
    
    return y_pred 

In [32]:
# Función para evaluar el MSE promedio y su desvío estándar en los distintos folds para cada lambda
def mse_table(model, elasticnet=None):
    inverse_Cs = 1 / model.Cs_
    results = []
    
    if elasticnet:
        l1_ratios = model.l1_ratios_
        for c_idx, lambda_ in enumerate(inverse_Cs):
            for l1_idx, l1_ratio in enumerate(l1_ratios):
                    mean_score = np.mean(-model.scores_[1][:, c_idx, l1_idx], axis=0)
                    std = np.std(-model.scores_[1][:, c_idx, l1_idx], axis=0)
                    results.append({
                        "Lambda": lambda_,
                        "L1 Ratio": l1_ratio,
                        "Mean MSE": mean_score,
                        "Std MSE": std
                    })
        
        results_table = pd.DataFrame(results).sort_values(by="Mean MSE", ascending=True)
    
    else:
        mean_scores = np.mean(-model.scores_[1], axis=0)
        std = np.std(-model.scores_[1], axis=0)
        results_table = pd.DataFrame({
            "Lambda": inverse_Cs,
            "Mean Score": mean_scores,
            "Std MSE": std
        }).sort_values(by="Mean Score", ascending=True)
    
    print(results_table)
    return results_table

In [33]:
# Función para quedarmos con los coeficientes que son distintos de 0
def non_zero_coefs(model, X_train):
    best_coefs = model.coef_[0]
    feature_names = X_train.columns
    non_zero_coefs = []
    for coef, name in zip(best_coefs, feature_names):
        if coef != 0:
            non_zero_coefs.append({
                "Variable": name,
                "Coeficiente": coef,
                'coef_abs': abs(coef)
            })
    non_zero_table = pd.DataFrame(non_zero_coefs).sort_values(by = 'coef_abs', ascending= False)
    print(non_zero_table.iloc[: , :2])
    
    return non_zero_table

In [34]:
prueba = data.loc[data['buen_historial'] == 1]
prueba = prueba.loc[prueba['default_lag'] == 1]

## LASSO

In [35]:
start = -3
end = 3

tscv = TimeSeriesSplit(n_splits=10) # Este cross validation tiene en cuenta la temporarlidad de la base

pipeline = Pipeline([
    ('scaler', StandardScaler()), # Primero estandariza los datos
    ('logreg', LogisticRegressionCV( # Estima el modelo usando cross validation para elegir el mejor hiperparámetro
        cv=tscv,
        penalty='l1',
        solver='saga',
        scoring='neg_mean_squared_error',
        max_iter=2000,
        random_state= 42,
        tol = 1e-3,
        n_jobs= -1,
        fit_intercept= True,
        Cs= np.logspace(start, end, 20)
    ))
])

del start, end

In [36]:
pipeline.fit(X_train, Y_train) # Entreno el modelo

Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegressionCV(Cs=array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12883789e+02, 2.33572147e+02, 4.83293024e+02, 1.00000000e+03]),
                                      cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
                                      max_iter=2000, n_jobs=-1, penalty='l1',
                                      random_state=42,
                                      scoring='neg_mean_squared_error',
                                      solver='saga', tol=0.001))])

In [37]:
joblib.dump(pipeline, './output/lasso_02_autorreg.pkl')

['./output/lasso_02_autorreg.pkl']

In [38]:
y_pred = eval(pipeline, X_test, Y_test)

[[162058     73]
 [   526  10625]]
El f1 es: 0.9725845576456589
El accuracy es: 0.9965432070266964
El MSE es: 0.0034567929733036323
El AUC es: 0.9761895447347647


In [39]:
lasso = pipeline.named_steps['logreg'] # Agarro el modelo desde el pipeline
best_c_lasso = lasso.C_[0]
print(f'El mejor lambda para el modelo es: {1/best_c_lasso}')

El mejor lambda para el modelo es: 1000.0


In [40]:
mse = mse_table(lasso)

         Lambda  Mean Score   Std MSE
0   1000.000000    0.020504  0.008617
1    483.293024    0.020504  0.008617
2    233.572147    0.023469  0.011436
3    112.883789    0.026103  0.017713
4     54.555948    0.026103  0.017713
5     26.366509    0.026103  0.017713
6     12.742750    0.026700  0.017426
7      6.158482    0.027125  0.017306
8      2.976351    0.028471  0.017393
19     0.001000    0.028484  0.017405
18     0.002069    0.028484  0.017405
10     0.695193    0.028490  0.017401
11     0.335982    0.028490  0.017401
12     0.162378    0.028490  0.017401
13     0.078476    0.028490  0.017401
14     0.037927    0.028490  0.017401
15     0.018330    0.028490  0.017401
16     0.008859    0.028490  0.017401
17     0.004281    0.028490  0.017401
9      1.438450    0.028490  0.017401


In [41]:
non_zero_lasso = non_zero_coefs(lasso, X_train)

                 Variable  Coeficiente
1               default_1     1.348171
2  prev_default_entidad_1     1.267726
3  prev_default_general_1     0.195046
0              sin_arca_1     0.081259


## Elastic Net

In [42]:
#start = -3
#end = 3

#tscv = TimeSeriesSplit(n_splits = 5)
#pipeline_en = Pipeline([
#    ('scaler', StandardScaler()),  
#    ('logreg', LogisticRegressionCV(
#        cv=tscv,
#        penalty='elasticnet',
#        solver='saga',
#        scoring='neg_mean_squared_error',
#        max_iter=2000,
#        random_state=42,
#        tol=1e-3,
#        n_jobs=-1,
#        fit_intercept=True,
#        Cs=np.linspace(start, end, 20), 
#        l1_ratios=np.linspace(0.1, 0.9, 9)  
#    ))
#])

#del start, end

In [43]:
#pipeline_en.fit(X_train, Y_train)

In [44]:
#joblib.dump(pipeline_en, './output/elasticnet_02.pkl')

In [45]:
#y_pred = eval(pipeline_en, X_test, Y_test)

In [46]:
#elasticnet = pipeline_en.named_steps['logreg']
#l1_ratio = elasticnet.l1_ratio_[0]
#print(f'La proporción óptima de LASSO es: {l1_ratio}')
#best_c_en = elasticnet.C_[0]
#print(f'El mejor lambda para el modelo es: {1/best_c_en}')

In [47]:
#mse = mse_table(elasticnet, True)

In [48]:
#non_zero_en = non_zero_coefs(elasticnet, X_train)

## RandomForest

In [49]:
rf = RandomForestClassifier()

tscv = TimeSeriesSplit(n_splits = 10)

param_dist = {
    'n_estimators': list(range(100, 701, 10)),  
    'max_depth': [None] + list(range(10, 71, 5)), 
}

halving_search = HalvingRandomSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    factor=3,  
    min_resources=10000,  
    resource="n_samples",  
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=4
)

In [50]:
halving_search.fit(X_train, Y_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 10000
max_resources_: 693125
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 69
n_resources: 10000
Fitting 10 folds for each of 69 candidates, totalling 690 fits
----------
iter: 1
n_candidates: 23
n_resources: 30000
Fitting 10 folds for each of 23 candidates, totalling 230 fits
----------
iter: 2
n_candidates: 8
n_resources: 90000
Fitting 10 folds for each of 8 candidates, totalling 80 fits
----------
iter: 3
n_candidates: 3
n_resources: 270000
Fitting 10 folds for each of 3 candidates, totalling 30 fits


HalvingRandomSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
                      estimator=RandomForestClassifier(), min_resources=10000,
                      n_jobs=-1,
                      param_distributions={'max_depth': [None, 10, 15, 20, 25,
                                                         30, 35, 40, 45, 50, 55,
                                                         60, 65, 70],
                                           'n_estimators': [100, 110, 120, 130,
                                                            140, 150, 160, 170,
                                                            180, 190, 200, 210,
                                                            220, 230, 240, 250,
                                                            260, 270, 280, 290,
                                                            300, 310, 320, 330,
                                                            340, 350, 360, 370,
                                                            380, 390, ...]},
                      random_state=42, scoring='neg_mean_squared_error',
                      verbose=4)

In [51]:
y_pred = eval(halving_search, X_test, Y_test)

[[162000    131]
 [   362  10789]]
El f1 es: 0.9776629966924925
El accuracy es: 0.9971549266513544
El MSE es: 0.0028450733486455604
El AUC es: 0.9833642776029603


In [52]:
joblib.dump(halving_search, './output/rf_02_autorreg.pkl')

['./output/rf_02_autorreg.pkl']

In [53]:
results_rf = pd.DataFrame(halving_search.cv_results_)

In [54]:
print(halving_search.best_params_)

{'n_estimators': 450, 'max_depth': 10}


In [55]:
rf = halving_search.best_estimator_

In [56]:
importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf.feature_importances_
}).sort_values(by='Importance', ascending= False)
print(importances)

                                               Feature  Importance
130                                          default_1    0.374785
131                             prev_default_entidad_1    0.251142
132                             prev_default_general_1    0.224045
129                                         sin_arca_1    0.078367
133                                   buen_historial_1    0.022732
..                                                 ...         ...
64   Tipo de Cambio Mayorista ($ por USD) Comunicac...    0.000007
102  BADLAR en pesos de bancos privados (en % n.a.)...    0.000007
110    Tasa de Política Monetaria (en % n.a.)_std_real    0.000005
50          Tasa de Política Monetaria (en % n.a.)_std    0.000004
48          Tasa de Política Monetaria (en % e.a.)_std    0.000002

[134 rows x 2 columns]


## Modelo de Probabilidad Lineal

In [57]:
pipeline_lm = Pipeline([
    ('scaler', StandardScaler()),
    ('lm', LinearRegression(fit_intercept= True, n_jobs= -1))
                       ])
                    
pipeline_lm.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('lm', LinearRegression(n_jobs=-1))])

In [58]:
joblib.dump(pipeline_lm, './output/lm_02_autorreg.pkl')

['./output/lm_02_autorreg.pkl']

In [59]:
y_pred = eval(pipeline_lm, X_test, Y_test, linear= True)

[[162058     73]
 [   526  10625]]
El f1 es: 0.9725845576456589
El accuracy es: 0.9965432070266964
El MSE es: 0.0034567929733036323
El AUC es: 0.9761895447347647


In [60]:
lm = pipeline_lm.named_steps['lm']

In [61]:
df_coeficientes = pd.DataFrame({
    'variable': X_train.columns,
    'coeficiente': lm.coef_,
    'abs_coef': np.abs(lm.coef_)
}).sort_values(by = 'abs_coef', ascending= False)
print(df_coeficientes.iloc[:,:2])

                                              variable   coeficiente
130                                          default_1  1.982040e-01
131                             prev_default_entidad_1  5.117583e-02
129                                         sin_arca_1  1.884032e-03
7                                        sin_historial -7.575630e-04
132                             prev_default_general_1  2.936885e-04
..                                                 ...           ...
50          Tasa de Política Monetaria (en % n.a.)_std  9.980814e-07
27   Depósitos en efectivo en las entidades financi...  9.231161e-07
78   Billetes y monedas en poder del público (en mi... -9.117776e-07
52         Tasa de interés de préstamos personales_std -3.011321e-07
49         Tasa de Política Monetaria (en % n.a.)_mean  2.016833e-07

[134 rows x 2 columns]


## Regresión Logística

In [62]:
pipeline_logit = Pipeline([
    ('scaler', StandardScaler()),
	('logit', LogisticRegression(max_iter = 2000, fit_intercept = True, n_jobs = -1, solver = 'saga'))
])
pipeline_logit.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('logit',
                 LogisticRegression(max_iter=2000, n_jobs=-1, solver='saga'))])

In [63]:
joblib.dump(pipeline_logit, './output/logit_02_autorreg.pkl')

['./output/logit_02_autorreg.pkl']

In [64]:
y_pred = eval(pipeline_logit, X_test, Y_test)

[[162058     73]
 [   526  10625]]
El f1 es: 0.9725845576456589
El accuracy es: 0.9965432070266964
El MSE es: 0.0034567929733036323
El AUC es: 0.9761895447347647


In [65]:
logit = pipeline_logit.named_steps['logit']

In [66]:
df_coeficientes = pd.DataFrame({
    'variable': X_train.columns,
    'coeficiente': logit.coef_[0],
    'abs_coef': np.abs(logit.coef_[0])
}).sort_values(by = 'abs_coef', ascending= False)
print(df_coeficientes.iloc[:, :2])

                                              variable  coeficiente
131                             prev_default_entidad_1     1.570795
130                                          default_1     1.412160
133                                   buen_historial_1     0.837939
132                             prev_default_general_1     0.563051
7                                        sin_historial    -0.226977
..                                                 ...          ...
51        Tasa de interés de préstamos personales_mean    -0.000223
47         Tasa de Política Monetaria (en % e.a.)_mean     0.000151
72   Índice para Contratos de Locación (ICL-Ley 27....    -0.000132
12   BADLAR en pesos de bancos privados (en % e.a.)...     0.000099
35     Inflación interanual (variación en % i.a.)_mean     0.000022

[134 rows x 2 columns]


In [67]:
pipeline = joblib.load('./output/lasso_02_autorreg.pkl')
#pipeline_en = joblib.load('./output/elasticnet_02.pkl')
halving_search = joblib.load('./output/rf_02_autorreg.pkl')
pipeline_lm = joblib.load('./output/lm_02_autorreg.pkl')
pipeline_logit = joblib.load('./output/logit_02_autorreg.pkl')

In [68]:
lasso = pipeline.named_steps['logreg']
best_c_lasso = lasso.C_[0]
#elasticnet = pipeline_en.named_steps['logreg']
#l1_ratio = elasticnet.l1_ratio_[0]
#best_c_en = elasticnet.C_[0]
rf = halving_search.best_estimator_
max_depth = rf.max_depth
n_estimators = rf.n_estimators

## VotingClassifier

In [69]:
logit_lasso = LogisticRegression(
    penalty="l1",
    C= best_c_lasso,  
    solver="saga",    
    random_state= 42
)

#logit_elastic = LogisticRegression(
#    penalty="elasticnet",
#    l1_ratio = l1_ratio, 
#    C= best_c_en,
#    solver="saga",
#    random_state= 42
#)

logit_no_reg = LogisticRegression(
    random_state= 42
)

rf = RandomForestClassifier(
    n_estimators = n_estimators,
    max_depth= max_depth,
    random_state= 42
)

estimadores = [
    ("logit_lasso", logit_lasso),
    ("logit_no_reg", logit_no_reg),
    ("random_forest", rf)
]

#### Soft

In [70]:
pipeline_voting = Pipeline([
	('scaler', StandardScaler()),
	('voting', VotingClassifier(
     estimators=estimadores,
     voting="soft",
     n_jobs = -1  
))])

In [71]:
tscv = TimeSeriesSplit(n_splits = 5)

param_distributions = {
    'voting__weights': [np.random.dirichlet(np.ones(len(estimadores))) for _ in range(80)]
}

halving_search_vote = HalvingRandomSearchCV(
    estimator=pipeline_voting,
    param_distributions=param_distributions,
    factor=3,  
    min_resources=10000,  
    resource="n_samples",
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    cv=tscv,
    random_state=42,
    verbose=4
)

In [72]:
halving_search_vote.fit(X_train, Y_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 10000
max_resources_: 693125
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 69
n_resources: 10000
Fitting 5 folds for each of 69 candidates, totalling 345 fits
----------
iter: 1
n_candidates: 23
n_resources: 30000
Fitting 5 folds for each of 23 candidates, totalling 115 fits
----------
iter: 2
n_candidates: 8
n_resources: 90000
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 3
n_candidates: 3
n_resources: 270000
Fitting 5 folds for each of 3 candidates, totalling 15 fits


HalvingRandomSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                      estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                                ('voting',
                                                 VotingClassifier(estimators=[('logit_lasso',
                                                                               LogisticRegression(C=0.001,
                                                                                                  penalty='l1',
                                                                                                  random_state=42,
                                                                                                  solver='saga')),
                                                                              ('logit_no_reg',
                                                                               LogisticRegression(random_state=42)),
                                                                              ('random_forest',
                                                                               Random...
                                                               array([0.70537772, 0.22780583, 0.06681645]),
                                                               array([0.00281801, 0.48099903, 0.51618296]),
                                                               array([0.56789983, 0.03781941, 0.39428076]),
                                                               array([0.11480231, 0.60965605, 0.27554164]),
                                                               array([0.34853701, 0.2304985 , 0.42096449]),
                                                               array([0.3070835 , 0.13781119, 0.5551053 ]),
                                                               array([0.34153901, 0.38528341, 0.27317758]), ...]},
                      random_state=42, scoring='neg_mean_squared_error',
                      verbose=4)

In [73]:
joblib.dump(halving_search_vote, './output/voting_02_autorreg.pkl')

['./output/voting_02_autorreg.pkl']

In [74]:
y_pred = eval(halving_search_vote, X_test, Y_test)

[[162058     73]
 [   526  10625]]
El f1 es: 0.9725845576456589
El accuracy es: 0.9965432070266964
El MSE es: 0.0034567929733036323
El AUC es: 0.9761895447347647


In [75]:
results_voting = pd.DataFrame(halving_search_vote.cv_results_)

In [76]:
voting = halving_search_vote.best_estimator_

# Sin Autorregresión

| Modelo             | TN     | FP  | FN  | TP    | F1       | Accuracy | MSE      | AUC      |
|--------------------|--------|-----|-----|-------|----------|----------|----------|----------|
| LASSO             | 161984 | 147 | 268 | 10883 | 0.981290 | 0.997605 | 0.002395 | 0.987530 |
| Random Forest     | 161984 | 147 | 265 | 10886 | 0.981428 | 0.997622 | 0.002378 | 0.987664 |
| Logit             | 161984 | 147 | 271 | 10880 | 0.981152 | 0.997588 | 0.002412 | 0.987395 |
| Linear Regression | 161983 | 148 | 264 | 10887 | 0.981430 | 0.997622 | 0.002378 | 0.987706 |
| Voting Classifier | 161985 | 146 | 268 | 10883 | 0.981335 | 0.997611 | 0.002389 | 0.987533 |


### Evaluado sobre las observaciones cuyo $default_t \neq default_{t+1}$

| Modelo             | TN   | FP  | FN  | TP   | F1       | Accuracy | MSE      | AUC      |
|--------------------|------|-----|-----|------|----------|----------|----------|----------|
| LASSO             | 1    | 72  | 264 | 262  | 0.609302 | 0.439066 | 0.560935 | 0.255899 |
| Random Forest     | 1    | 72  | 265 | 261  | 0.607683 | 0.437396 | 0.562604 | 0.254948 |
| Logit             | 2    | 71  | 266 | 260  | 0.606768 | 0.437396 | 0.562604 | 0.260847 |
| Linear Regression | 0    | 73  | 264 | 262  | 0.608595 | 0.437396 | 0.562604 | 0.249049 |
| Voting Classifier | 2    | 71  | 265 | 261  | 0.608392 | 0.439066 | 0.560935 | 0.261797 |


# Con Autorregresión

| Modelo             | TP    | TN      | FP  | FN  | F1-Score  | Accuracy  | MSE       | AUC        |
|--------------------|-------|---------|-----|-----|-----------|-----------|-----------|------------|
| LASSO             | 10,625 | 162,058 | 73  | 526 | 0.9726    | 0.9965    | 0.0035    | 0.9762     |
| Random Forest     | 10,789 | 162,000 | 131 | 362 | 0.9777    | 0.9972    | 0.0028    | 0.9834     |
| Logit             | 10,625 | 162,058 | 73  | 526 | 0.9726    | 0.9965    | 0.0035    | 0.9762     |
| Linear Regression | 10,625 | 162,058 | 73  | 526 | 0.9726    | 0.9965    | 0.0035    | 0.9762     |
| Voting Classifier | 10,625 | 162,058 | 73  | 526 | 0.9726    | 0.9965    | 0.0035    | 0.9762     |

### Evaluado sobre las observaciones cuyo $default_t \neq default_{t+1}$

| Modelo             | TP  | TN | FP  | FN  | F1-Score  | Accuracy | MSE  | AUC  |
|--------------------|-----|----|----|----|-----------|----------|------|------|
| LASSO             | 0   | 0  | 73 | 526 | 0.0000    | 0.0000   | 1.0000 | 0.0000 |
| Random Forest     | 164 | 0  | 73 | 362 | 0.4299    | 0.2738   | 0.7262 | 0.1559 |
| Logit             | 0   | 0  | 73 | 526 | 0.0000    | 0.0000   | 1.0000 | 0.0000 |
| Linear Regression | 0   | 0  | 73 | 526 | 0.0000    | 0.0000   | 1.0000 | 0.0000 |
| Voting Classifier | 0   | 0  | 73 | 526 | 0.0000    | 0.0000   | 1.0000 | 0.0000 |


In [77]:
print('Cantidad de Créditos que pasaron de estar defaulteados a estar al día:')
print(len(Y_test_cambio.loc[Y_test_cambio == 0]))
print('Cantidad de créditos que pasaron de estar al día a estar defaulteados:')
print(len(Y_test_cambio.loc[Y_test_cambio == 1]))

Cantidad de Créditos que pasaron de estar defaulteados a estar al día:
73
Cantidad de créditos que pasaron de estar al día a estar defaulteados:
526


In [78]:
print('Proporción de créditos al día:')
print(len(Y_test.loc[Y_test == 0])/len(Y_test))

Proporción de créditos al día:
0.9356482496739419


Un predictor trivial que siempre prediga que el crédito no va a estar defaulteado le pegaría el 93.56% de las veces. Para los créditos que en $t$ estaban en un estado y en $t+1$ pasaron a estar en otro, este predictor trivial le pegaría un 12.07% de las veces

In [81]:
print('LASSO:')
y_pred = eval(pipeline, X_test, Y_test)
#print('Elastic Net:')
#y_pred = eval(pipeline_en, X_test, Y_test)
print('Random Forest:')
y_pred = eval(halving_search, X_test, Y_test)
print('Logit:')
y_pred = eval(pipeline_logit, X_test, Y_test)
print('Linear Regression:')
y_pred = eval(pipeline_lm, X_test, Y_test, linear = True)
print('Voting Classifier:')
y_pred = eval(halving_search_vote, X_test, Y_test)

LASSO:
[[162058     73]
 [   526  10625]]
El f1 es: 0.9725845576456589
El accuracy es: 0.9965432070266964
El MSE es: 0.0034567929733036323
El AUC es: 0.9761895447347647
Random Forest:
[[162000    131]
 [   362  10789]]
El f1 es: 0.9776629966924925
El accuracy es: 0.9971549266513544
El MSE es: 0.0028450733486455604
El AUC es: 0.9833642776029603
Logit:
[[162058     73]
 [   526  10625]]
El f1 es: 0.9725845576456589
El accuracy es: 0.9965432070266964
El MSE es: 0.0034567929733036323
El AUC es: 0.9761895447347647
Linear Regression:
[[162058     73]
 [   526  10625]]
El f1 es: 0.9725845576456589
El accuracy es: 0.9965432070266964
El MSE es: 0.0034567929733036323
El AUC es: 0.9761895447347647
Voting Classifier:
[[162058     73]
 [   526  10625]]
El f1 es: 0.9725845576456589
El accuracy es: 0.9965432070266964
El MSE es: 0.0034567929733036323
El AUC es: 0.9761895447347647


In [82]:
print('LASSO:')
y_pred = eval(pipeline, X_test_cambio, Y_test_cambio)
#print('Elastic Net:')
#y_pred = eval(pipeline_en, X_test_cambio, Y_test_cambio)
print('Random Forest:')
y_pred = eval(halving_search, X_test_cambio, Y_test_cambio)
print('Logit:')
y_pred = eval(pipeline_logit, X_test_cambio, Y_test_cambio)
print('Linear Regression:')
y_pred = eval(pipeline_lm, X_test_cambio, Y_test_cambio, linear = True)
print('Voting Classifier:')
y_pred = eval(halving_search_vote, X_test_cambio, Y_test_cambio)

LASSO:
[[  0  73]
 [526   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0
Random Forest:
[[  0  73]
 [362 164]]
El f1 es: 0.42988204456094364
El accuracy es: 0.27378964941569284
El MSE es: 0.7262103505843072
El AUC es: 0.155893536121673
Logit:
[[  0  73]
 [526   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0
Linear Regression:
[[  0  73]
 [526   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0
Voting Classifier:
[[  0  73]
 [526   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0
